In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmasher as cmr
import numpy as np
import glob
from scipy.stats import linregress
from decimal import Decimal, ROUND_DOWN
import statsmodels.api as sm
import statsmodels.formula.api as smf

/home/estudiantes/anaconda/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df=pd.read_csv("/home/andreagvc/data/ERA/ERA/IDD.csv",header=None) #Ruta del archivo csv con los metadatos de las estaciones globales de HadISD
ID=df[(df[0]>="760011-99999")&(df[0]<="780160-13601")]  #Seleccionando las estaciones dentro de México
ID1=ID[0].str.split(pat="-", expand=True) #Separando 
ID["ID"]=ID1[0].astype(int)

esta=pd.read_fwf("/home/andreagvc/data/ERA/ERA/ID.txt",header=None) #Ruta del archivo que contiene el id de 
                                                          #las estaciones restantes después del análisis de calidad de los datos
p=esta[1]
filtered_df = ID[(ID["ID"]).isin(p)] #Filtrando las 90 estaciones con los metadatos para sustraer su latitud, longitud y elevación.
filtered_df #Encontrando los datos de las estaciones que ocupamos
lat=filtered_df[2].values
lon=filtered_df[3].values
ele=filtered_df[4].values

#Agregando la elevación al dataframe
esta["elevacion"]=ele
esta["Lat"]=lat
esta["Lon"]=lon
esta["Lugar"]=filtered_df[1].values

/tmp/ipykernel_175313/1441987256.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID["ID"]=ID1[0].astype(int)


In [ ]:
# Usar glob para seleccionar todos los archivos CSV en la ruta
archivos_csv = glob.glob("/home/andreagvc/data/ERA/ERA/aux_csv/*.csv") #Recorte de ERA5

# Leer y concatenar todos los archivos CSV
df_concatenado = pd.concat([pd.read_csv(f) for f in archivos_csv], ignore_index=True)

# Ver el DataFrame concatenado
print(df_concatenado)

In [ ]:
fig = plt.figure(figsize=(15, 4),dpi=300)
plt.subplot(1,3,1)
plt.scatter(T["HadSID"],T["ERA5"],color="orange")
plt.ylabel("ERA5-Land [$^\circ$C]",fontsize=13)
plt.text(27.5, 13, "a) $T$" , fontsize=15, color="black",
            ha="left", va="top", bbox=dict(facecolor="lightgray", edgecolor="black",
                                           boxstyle="round", pad=0.4))

# Preparar datos
x_raw = np.asarray(T["HadSID"])
y_raw = np.asarray(T["ERA5"])

# Get sort indices from 'a'
idx = np.argsort(x_raw)

# Sort both arrays using the same indices
x_raw_sorted = x_raw[idx]
y_raw_sorted = y_raw[idx]
# Ajustar modelo lineal
X = sm.add_constant(x_raw_sorted)
    # Ajustar el modelo de regresión lineal utilizando OLS (Mínimos cuadrados ordinarios)
modelo = sm.OLS(y_raw_sorted, X).fit()
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(14,29,(f" $R^2{{:}} {modelo.rsquared:.2f}$"),fontsize=14,bbox=props)
# Graficar los datos y la línea de regresión
plt.scatter(x_raw, y_raw,color="orange")
plt.plot(x_raw_sorted, modelo.predict(X), color='black')
plt.fill_between(x_raw_sorted, modelo.predict(X), 30, color="red", alpha=0.09, label="Sobrestima")
plt.fill_between(x_raw_sorted, 12, modelo.predict(X), color="blue", alpha=0.09, label="Subestima")
plt.grid(color='gray', alpha=0.5, linestyle='--')
plt.legend(bbox_to_anchor=(0,-0.3), loc='lower left',ncol=2,fontsize=14)
plt.xticks(fontsize=14)
plt.yticks((np.arange(12,31,2)),fontsize=14)
plt.xlabel("HadISD [$^\circ$C]",fontsize=13)

plt.subplot(1,3,2)

plt.scatter(Tw["HadSID"]-273.15,Tw["ERA5"],color="r")
x_raw=np.asarray(Tw["HadSID"]-273.15)
y_raw = np.asarray(Tw["ERA5"])

# Get sort indices from 'a'
idx = np.argsort(x_raw)

# Sort both arrays using the same indices
x_raw_sorted = x_raw[idx]
y_raw_sorted = y_raw[idx]
# Ajustar modelo lineal
X = sm.add_constant(x_raw_sorted)
    # Ajustar el modelo de regresión lineal utilizando OLS (Mínimos cuadrados ordinarios)
modelo = sm.OLS(y_raw_sorted, X).fit()
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# Graficar los datos y la línea de regres
plt.text(9.5,24,(f" $R^2{{:}} {modelo.rsquared:.2f}$"),fontsize=14,bbox=props)
plt.plot(x_raw_sorted, modelo.predict(X), color='black')
plt.fill_between(x_raw_sorted, modelo.predict(X), 25, color="red", alpha=0.09, label="Sobrestima")
plt.fill_between(x_raw_sorted, 5, modelo.predict(X), color="blue", alpha=0.09, label="Subestima")
plt.ylabel("ERA5-Land [$^\circ$C]",fontsize=13)
plt.xlabel("HadISD [$^\circ$C]",fontsize=13)
plt.grid(color='gray', alpha=0.5, linestyle='--')
plt.xticks((np.arange(9,27,3)),fontsize=14)
plt.yticks((np.arange(6,27,3)),fontsize=14)
plt.text(22.7, 6.3, "b) $T_{W}$" , fontsize=15, color="black",
            ha="left", va="top", bbox=dict(facecolor="lightgray", edgecolor="black",
                                           boxstyle="round", pad=0.4))

plt.subplot(1,3,3)

x_raw=np.asarray(q["HadSID"]*1000)
y_raw = np.asarray(q["ERA5"]*1000)

# Get sort indices from 'a'
idx = np.argsort(x_raw)

# Sort both arrays using the same indices
x_raw_sorted = x_raw[idx]
y_raw_sorted = y_raw[idx]
# Ajustar modelo lineal
X = sm.add_constant(x_raw_sorted)
    # Ajustar el modelo de regresión lineal utilizando OLS (Mínimos cuadrados ordinarios)
modelo = sm.OLS(y_raw_sorted, X).fit()
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
# Graficar los datos y la línea de regres
plt.text(6,17.3,(f" $R^2{{:}} {modelo.rsquared:.2f}$"),fontsize=14,bbox=props)
plt.plot(x_raw_sorted, modelo.predict(X), color='black')
plt.fill_between(x_raw_sorted, modelo.predict(X), 18, color="red", alpha=0.09, label="Sobrestima")
plt.fill_between(x_raw_sorted, 5, modelo.predict(X), color="blue", alpha=0.09, label="Subestima")

plt.scatter(q["HadSID"]*1000,q["ERA5"]*1000)

plt.ylabel("ERA5-Land [$g$ $kg^{-1}$]",fontsize=13)
plt.xlabel("HadISD [$g$ $kg^{-1}$]",fontsize=13)
plt.text(16.6, 5.8, "c) $SH$", fontsize=15, color="black",
            ha="left", va="top", bbox=dict(facecolor="lightgray", edgecolor="black",
                                           boxstyle="round", pad=0.4))
plt.grid(color='gray', alpha=0.5, linestyle='--')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout(h_pad=-1,pad=-2)
plt.savefig("ERAvsHAD.svg",dpi=300,bbox_inches="tight")